Importing our libraries!

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone

In [ ]:
from bs4 import BeautifulSoup
!pip install lat-lon-parser
from lat_lon_parser import parse

In [288]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Accessing data from Rapid API 

In [ ]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

querystring = {"lat":"37.77","lon":"-122.41","radiusKm":"50","limit":"10","withFlightInfoOnly":"true"}

headers = {
	"x-rapidapi-key": #ENTER YOUR API KEY,
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

In [ ]:
response.json()

Exploring the data we just got

In [ ]:
pd.json_normalize(response.json()['items'])

Very important function to get the airports for our cities: Hamburg, Munich, San Francisco, and La Paz.

In [ ]:
def icao_airport_codes(latitudes, longitudes):

  list_for_df = []

  for index, value in enumerate(latitudes):

    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{value}/{longitudes[index]}/km/100/16"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
      "X-RapidAPI-Host": #ENTER YOUR API KEY,
      "X-RapidAPI-Key": "687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    list_for_df.append(pd.json_normalize(response.json()['items']))

  return pd.concat(list_for_df, ignore_index=True)

Now lets input our coordinates into the function and turn it into a data frame

In [ ]:
# coordinates for hamburg, munich, SF, LP
latitudes = [53.55, 48.1375, 37.7775, -16.4958]
longitudes = [10.00, 11.575, -122.4163, -68.1333]

airp= icao_airport_codes(latitudes, longitudes)
my_port_df= pd.DataFrame(airp) 

In [ ]:
my_port_df  #very important table!

We need to create a table from the above for only airports with name and icao

In [ ]:
airports =my_port_df[["icao", 
                          "iata", 
                          "name" , 
                          "countryCode"]]
airports#to be moved to SQL

we need a table for city_airport with city_id(from citiees in sql), city name, icao

In [ ]:
city_airport= my_port_df[["name" , 
                          "icao" , 
                          "iata"]]
city_airport #move to SQL add city_id from cities in sql

Let's access data from AeroDataBox API and get the information on flights arriving to our cities

In [ ]:
#TAKING HAM for hamburg(maybe we can repalce this for our loop?) my_port_df  table is very important for this loop
i = "HAM"
date = datetime.now().date()
time_1 = "00:00"
time_2 = "11:59"

url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{i}/{date}T{time_1}/{date}T{time_2}"

querystring = {"withLeg":"true","direction":"both","withCancelled":"false",
               "withCodeshared":"true","withCargo":"false","withPrivate":"false",
               "withLocation":"true"}

headers = {
	"x-rapidapi-key": #ENTER YOUR API KEY,
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

rspo= (pd.json_normalize(response.json()['arrivals']))
rspo[["number", 
      "status", 
      "departure.airport.name" , 
      "departure.airport.icao",
      "arrival.scheduledTime.local", 
      "arrival.terminal", 
      "airline.name"]]


We take into account the cities in Germany only as the info for LPB is not the same as the rest cities.

In [357]:
airports = ["HAM", "MUC", "SFO"]
date = datetime.now().date()
time_1 = "00:00"
time_2 = "11:59"
flights=[]

for i in airports:
        
    
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{i}/{date}T{time_1}/{date}T{time_2}"
    
    querystring = {"withLeg":"true","direction":"both","withCancelled":"false",
                   "withCodeshared":"true","withCargo":"false","withPrivate":"false",
                   "withLocation":"true"}
    
    headers = {
    	"x-rapidapi-key": #ENTER YOUR API KEY,
    	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    
    rspo= (pd.json_normalize(response.json()['arrivals']))
    flights.append(rspo[["number", 
                         "airline.iata",
                         "airline.icao",
                          "departure.airport.name" , 
                          "departure.airport.icao",
                         "arrival.scheduledTime.local",
                          "arrival.terminal", 
                          "airline.name"]])
                   


In [359]:
flights_df = pd.concat(flights, ignore_index=True)
flights_df.head(5)

,number,airline.iata,airline.icao,departure.airport.name,departure.airport.icao,arrival.scheduledTime.local,arrival.terminal,airline.name
0,EW 2028,EW,EWG,Stuttgart,EDDS,2024-12-04 07:45+01:00,1,Eurowings
1,EW 9030,EW,EWG,Düsseldorf,EDDL,2024-12-04 07:45+01:00,1,Eurowings
2,KL 1751,KL,KLM,Amsterdam,EHAM,2024-12-04 07:55+01:00,1,KLM
3,LH 2050,LH,DLH,Munich,EDDM,2024-12-04 08:00+01:00,2,Lufthansa
4,EW 7033,EW,EWG,Cologne,EDDK,2024-12-04 08:00+01:00,1,Eurowings
5,LH 2,LH,DLH,Frankfurt-am-Main,EDDF,2024-12-04 07:35+01:00,2,Lufthansa
6,FR 2050,FR,RYR,Palma De Mallorca,LEPA,2024-12-04 08:35+01:00,2,Ryanair
7,LX 1046,LX,SWR,Geneva,LSGG,2024-12-04 08:40+01:00,2,SWISS
8,LX 1050,LX,SWR,Zurich,LSZH,2024-12-04 08:45+01:00,2,SWISS
9,OS 175,OS,AUA,Vienna,LOWW,2024-12-04 08:55+01:00,2,Austrian


In [363]:
flights_df['arrival.scheduledTime.local'] = flights_df['arrival.scheduledTime.local'].str[:-6]

In [367]:
flights_df.head(5)

,number,airline.iata,airline.icao,departure.airport.name,departure.airport.icao,arrival.scheduledTime.local,arrival.terminal,airline.name
0,EW 2028,EW,EWG,Stuttgart,EDDS,2024-12-04 07:45,1,Eurowings
1,EW 9030,EW,EWG,Düsseldorf,EDDL,2024-12-04 07:45,1,Eurowings
2,KL 1751,KL,KLM,Amsterdam,EHAM,2024-12-04 07:55,1,KLM
3,LH 2050,LH,DLH,Munich,EDDM,2024-12-04 08:00,2,Lufthansa
4,EW 7033,EW,EWG,Cologne,EDDK,2024-12-04 08:00,1,Eurowings


NOW LETS transfer our three tables to SQL!!

FOR SQL 

-- Use the database
USE gans_local;

airports =my_port_df[["icao", 
                          "iata", 
                          "name" , 
                          "countryCode"]]
airports #to be moved to SQL

city_airport= my_port_df[["name" , 
                          "icao" , 
                          "iata"]]
city_airport #move to SQL add city_id from cities in sql

flights.append(rspo[["number", 
                         "airline.iata",
                         "airline.icao",
                          "departure.airport.name" , 
                          "departure.airport.icao",
                          "arrival.scheduledTime.local", 
                          "arrival.terminal", 
                          "airline.name"]])


CREATE TABLE aiports (
    airports_id INT AUTO_INCREMENT, -- Automatically generated ID for each aiport
    icao VARCHAR(25) NOT NULL, 
    iata VARCHAR(25) NOT NULL, 
    City_name VARCHAR(255) NOT NULL, -- Name of the city
    City_id INT NOT NULL,
    PRIMARY KEY (icao) -- Primary key to uniquely identify each city
    FOREIGN KEY (City_id) REFERENCES cities(City_id)
);



CREATE TABLE city_airport (
    city_airport_id INT AUTO_INCREMENT,    
    Name VARCHAR(255) NOT NULL, -- Name of the city
    icao VARCHAR(25) NOT NULL, 
    iata VARCHAR(25) NOT NULL, 
    City_id INT,
    PRIMARY KEY (city_airport_id),
    FOREIGN KEY (iata) REFERENCES aiports(iata),
    FOREIGN KEY (city_id) REFERENCES cities(city_id)
)


CREATE TABLE flights(
    flight.id INT AUTO_INCREMENT,
    number VARCHAR (50)
    airline.iata VARCHAR(10),
    airline.icao VARCHAR(10),
    departure.airport.name VARCHAR(30),
    departure.airport.icao VARCHAR(30),
    arrival.scheduledTime.local DATETIME,
    arrival.terminal VARCHAR(30),
    airline.name VARCHAR(255),
    PRIMARY KEY (flight.id),
    FOREIGN KEY (departure.airport.icao) REFERENCES airports(icao)
);

Passing airports to SQL

In [277]:
schema = "gans_local" # The name of your database
host = "127.0.0.1"
user = "root"
password = # USE Your MySQL password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [279]:
cities_from_sql = pd.read_sql("cities", con=connection_string)
cities_from_sql

,City_id,City_name,Country
0,1,Hamburg,Germany
1,2,Munich,Germany
2,3,San Francisco,United States
3,4,La Paz,Bolivia


Matching our tables to MySQL

In [ ]:
airports.head(5)

In [ ]:
#df.rename(columns={'oldName1': 'newName1'

airports_df=airports.rename(columns= { "name" : "City_name"})
airports_df.head(5)


In [ ]:
#df = df.drop('column_name', axis=1)
airports_df= airports_df.drop({"countryCode"} ,axis=1)
airports_df.head(5)

In [ ]:
#send airports table to SQL
airports_df.to_sql("airports",
                  if_exists="append",
                  con=connection_string,
                  index=False)

In [ ]:
What about city_id for airports

In [ ]:
airports_merged = pd.merge(airports_df,cities_from_sql, on ="City_name")
airports_merged                                 
#pd.merge(df1, df2, on='company')

Sending city_airport to SQL

In [ ]:
city_airport.head(5)

In [211]:
#df.rename(columns={'oldName1': 'newName1'

city_airport_df=city_airport.rename(columns= { "name" : "City_name"})
city_airport_df.head(5)


,City_name,icao,iata
0,Hamburg,EDDH,HAM
1,Bremen,EDDW,BRE
2,Munich,EDDM,MUC
3,Innsbruck,LOWI,INN
4,San Francisco,KSFO,SFO


In [ ]:
cities_from_sql = pd.read_sql("cities", con=connection_string)
cities_from_sql

merging the "cities_airports" and airports to get them the IDs

In [ ]:
cities_airports_merged = city_airport_df.merge(cities_from_sql,
                                   on = "City_name",
                                   how="left")

In [ ]:
cities_airports_merged2= city_airport_df.merge(cities_from_sql, how= "left", on="City_name") 
cities_airports_merged2.head(5)

In [ ]:
cities_airports_merged


In [213]:
#send airports table to SQL
city_airport_df.to_sql("city_airport",
                  if_exists="append",
                  con=connection_string,
                  index=False)

17

In [345]:
cities_airport_sql = pd.read_sql("city_airport", con=connection_string)
cities_airport_sql.head(5)

,city_airport_id,City_name,icao,iata,City_id
0,1,Hamburg,EDDH,HAM,None
1,2,Bremen,EDDW,BRE,None
2,3,Munich,EDDM,MUC,None
3,4,Innsbruck,LOWI,INN,None
4,5,San Francisco,KSFO,SFO,None


lets pass flights to sql

In [369]:
flights_df.head(5)

,number,airline.iata,airline.icao,departure.airport.name,departure.airport.icao,arrival.scheduledTime.local,arrival.terminal,airline.name
0,EW 2028,EW,EWG,Stuttgart,EDDS,2024-12-04 07:45,1,Eurowings
1,EW 9030,EW,EWG,Düsseldorf,EDDL,2024-12-04 07:45,1,Eurowings
2,KL 1751,KL,KLM,Amsterdam,EHAM,2024-12-04 07:55,1,KLM
3,LH 2050,LH,DLH,Munich,EDDM,2024-12-04 08:00,2,Lufthansa
4,EW 7033,EW,EWG,Cologne,EDDK,2024-12-04 08:00,1,Eurowings


In [371]:
#df.rename(columns={'oldName1': 'newName1'

flights_df=flights_df.rename(columns= {"departure.airport.icao" : "departure_airport_icao", "airline.iata" : "airline_iata","airline.icao" : "airline_icao", "departure.airport.name" : "departure_airport_name", "arrival.scheduledTime.local" : "arrival_scheduledTime_local", "arrival.terminal" : "arrival_terminal" , "airline.name" : "airline_name" })
flights_df.head(5)


,number,airline_iata,airline_icao,departure_airport_name,departure_airport_icao,arrival_scheduledTime_local,arrival_terminal,airline_name
0,EW 2028,EW,EWG,Stuttgart,EDDS,2024-12-04 07:45,1,Eurowings
1,EW 9030,EW,EWG,Düsseldorf,EDDL,2024-12-04 07:45,1,Eurowings
2,KL 1751,KL,KLM,Amsterdam,EHAM,2024-12-04 07:55,1,KLM
3,LH 2050,LH,DLH,Munich,EDDM,2024-12-04 08:00,2,Lufthansa
4,EW 7033,EW,EWG,Cologne,EDDK,2024-12-04 08:00,1,Eurowings


In [387]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 718 entries, 0 to 732
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   number                       718 non-null    object
 1   airline_iata                 718 non-null    object
 2   airline_icao                 718 non-null    object
 3   departure_airport_name       718 non-null    object
 4   departure_airport_icao       718 non-null    object
 5   arrival_scheduledTime_local  718 non-null    object
 6   arrival_terminal             718 non-null    object
 7   airline_name                 718 non-null    object
dtypes: object(8)
memory usage: 50.5+ KB


In [383]:
flights_df.departure_airport_icao.isna().sum()

2

In [385]:
flights_df= flights_df.loc[~flights_df. departure_airport_icao.isna(), :]

In [389]:
flights_df


,number,airline_iata,airline_icao,departure_airport_name,departure_airport_icao,arrival_scheduledTime_local,arrival_terminal,airline_name
0,EW 2028,EW,EWG,Stuttgart,EDDS,2024-12-04 07:45,1,Eurowings
1,EW 9030,EW,EWG,Düsseldorf,EDDL,2024-12-04 07:45,1,Eurowings
2,KL 1751,KL,KLM,Amsterdam,EHAM,2024-12-04 07:55,1,KLM
3,LH 2050,LH,DLH,Munich,EDDM,2024-12-04 08:00,2,Lufthansa
4,EW 7033,EW,EWG,Cologne,EDDK,2024-12-04 08:00,1,Eurowings
5,LH 2,LH,DLH,Frankfurt-am-Main,EDDF,2024-12-04 07:35,2,Lufthansa
6,FR 2050,FR,RYR,Palma De Mallorca,LEPA,2024-12-04 08:35,2,Ryanair
7,LX 1046,LX,SWR,Geneva,LSGG,2024-12-04 08:40,2,SWISS
8,LX 1050,LX,SWR,Zurich,LSZH,2024-12-04 08:45,2,SWISS
9,OS 175,OS,AUA,Vienna,LOWW,2024-12-04 08:55,2,Austrian


In [393]:
#send flights table to SQL
flights_df.to_sql('flights',
                    if_exists='append',
                    con=connection_string,
                    index=False)

718

In [397]:
flights_df_sql = pd.read_sql("flights", con=connection_string)
flights_df_sql.head(5)

,flight_id,number,airline_iata,airline_icao,departure_airport_name,departure_airport_icao,arrival_scheduledTime_local,arrival_terminal,airline_name
0,1,EW 2028,EW,EWG,Stuttgart,EDDS,2024-12-04 07:45:00,1,Eurowings
1,2,EW 9030,EW,EWG,Düsseldorf,EDDL,2024-12-04 07:45:00,1,Eurowings
2,3,KL 1751,KL,KLM,Amsterdam,EHAM,2024-12-04 07:55:00,1,KLM
3,4,LH 2050,LH,DLH,Munich,EDDM,2024-12-04 08:00:00,2,Lufthansa
4,5,EW 7033,EW,EWG,Cologne,EDDK,2024-12-04 08:00:00,1,Eurowings
